In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# RNN model
class ntwkRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout, bidirectional, select):
        super(ntwkRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)

        self.RNN = nn.RNN(hidden_size, hidden_size, batch_first=True, dropout = dropout, bidirectional=bidirectional)
        
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, select):
        embedded = self.embedding(x)
        output, _ = self.RNN(embedded)
        output = self.fc(output[:, -1, :])  #last output of RNN
        return output

class ntwkGRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout, bidirectional):
        super(ntwkRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        
        self.GRU = nn.GRU(hidden_size, hidden_size, batch_first=True, dropout = dropout, bidirectional=bidirectional)
        
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.GRU(embedded)
        output = self.fc(output[:, -1, :])  #last output of RNN
        return output
    
class ntwkLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout, bidirectional):
        super(ntwkRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        
        self.LSTM = nn.LSTM(hidden_size, hidden_size, batch_first=True, dropout = dropout, bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.LSTM(embedded)
        output = self.fc(output[:, -1, :])  #last output of RNN
        return output

In [1]:
#training loop for RNN model
def train_RNN(epochs, model, criterion, optimizer, train_loader, test_loader, device):
    train_loss = []
    val_loss = []
    val_acc = []
    for epoch in range(epochs):
        for seq, labels in train_loader:
            seq = seq.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            y_pred = model(seq)
            loss = criterion(y_pred, labels)
            loss.backward()
            optimizer.step()
            
        train_loss.append(loss)
        
        with torch.no_grad():
            for seq, labels in test_loader:
                seq = seq.to(device)
                labels = labels.to(device)
                
                val_output = model(seq)
                val_loss = criterion(val_output, labels)
                _, predicted = torch.max(val_output, 1)
                val_accuracy = (predicted == labels).float().mean()
                
        val_loss.append(loss)
        val_acc.append(val_accuracy)
        if (epoch+1) % 10 == 0:
            print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy.item()}')
            
    return train_loss, val_loss, val_acc
    

In [4]:
#transformer model
class ntwkTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, nheads, dropout):
        super(ntwkTransformer, self).__init__()
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nheads, dropout = dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        transformer_output = self.transformer_encoder(embedded)
        output = self.fc(transformer_output[:, -1, :])  # Get the output of the last Transformer block
        return output

In [5]:
#Posintional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        return x + self.encoding[:, :x.size(1)].detach()

#transformer w/ Positional Encoding for regression
class ntwkPETransformer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, nhead):
        super(ntwkPETransformer, self).__init__()
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.pos_encoder = PositionalEncoding(hidden_size)
        
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=2) 


    def forward(self, x):
        embedded = self.embedding(x)
        embedded = self.pos_encoder(embedded)
        
        transformer_output = self.transformer_encoder(embedded)
        output = self.fc(transformer_output)
        
        return output

In [6]:
#train loop for transformer
def trainTransformer(model,epochs, criterion, optimizer, train_loader, test_loader, device):
    train_loss = []
    val_loss = []
    val_acc = []
    for epoch in range(epochs):
        for seq, labels in train_loader:
            optimizer.zero_grad()
            
            seq = seq.to(device)
            output = model(seq)
            
            loss = criterion(output.transpose(1, 2), labels)
            loss.backward()
            optimizer.step()
        
        train_loss.append(loss)
        
        with torch.no_grad():
            for seq, labels in test_loader:
                seq = seq.to(device)
                labels = labels.to(device)
                val_output = model(seq)
                val_loss = criterion(val_output, labels)
                _, predicted = torch.max(val_output, 1)
                val_accuracy = (predicted == labels).float().mean()
                
            val_loss.append(loss)
            val_acc.append(val_accuracy)
        if (epoch+1) % 10 == 0:
            print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy.item()}')
            
    return train_loss, val_loss, val_acc

In [ ]:
def lossplot(t_loss,v_loss, epochs):
    
    plt.plot(t_loss,epochs, label="Training Loss")
    plt.plot(v_loss,epochs, label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Training and Validation Loss Over Epochs")
    plt.show()